In [1]:
import pandas as pd
import numpy as np
car = pd.read_csv('cars.csv')
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [2]:
car.shape


(892, 6)

In [3]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 21.0+ KB


In [4]:
backup=car.copy()
car = car[car['year'].str.isnumeric()]
car['year'].astype(int)

0      2007
1      2006
2      2018
3      2014
4      2014
       ... 
886    2009
888    2018
889    2013
890    2014
891    2014
Name: year, Length: 842, dtype: int32

In [5]:
car = car[car['Price']!="Ask For Price"]
car['Price']=car['Price'].str.replace(',','').astype(int)

In [6]:
car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',',' ')

In [7]:
car=car[car['kms_driven'].str.isnumeric()]
car['kms_driven']=car['kms_driven'].astype(int)

In [8]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 1 to 762
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        17 non-null     object
 1   company     17 non-null     object
 2   year        17 non-null     object
 3   Price       17 non-null     int32 
 4   kms_driven  17 non-null     int32 
 5   fuel_type   17 non-null     object
dtypes: int32(2), object(4)
memory usage: 544.0+ bytes


In [9]:
car=car[~car['fuel_type'].isna()]

In [10]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(" ")


In [11]:
car=car.reset_index(drop=True)

In [12]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Mahindra Jeep MM,Mahindra,2019,390000,60,Diesel
3,Mahindra Jeep CL550,Mahindra,2018,385000,588,Diesel
4,Maruti Suzuki Swift,Maruti,2019,610000,73,Petrol


In [13]:
car=car[car['Price']<6e6].reset_index(drop=True)

In [14]:
car.to_csv('Cleaned_car.csv')

In [15]:
x=car.drop(columns='Price')
y=car['Price']

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split( x,y,test_size=0.2)

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


In [18]:
ohe=OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder()

In [19]:
ohe.categories_

[array(['Ford Figo Petrol', 'Hyundai Creta 1.6', 'Mahindra Jeep CL550',
        'Mahindra Jeep MM', 'Maruti Suzuki Swift', 'Maruti Suzuki Zen',
        'Nissan X Trail', 'Tata Tiago Revotorq', 'Tata Tigor Revotron',
        'Toyota Innova 2.5'], dtype=object),
 array(['Ford', 'Hyundai', 'Mahindra', 'Maruti', 'Nissan', 'Tata',
        'Toyota'], dtype=object),
 array(['Diesel', 'Petrol'], dtype=object)]

In [20]:
column_trans = make_column_transformer((OneHotEncoder(categories = ohe.categories_),['name','company','fuel_type']),
                                       remainder = 'passthrough')

In [21]:
lr = LinearRegression()

In [22]:
pipe=make_pipeline(column_trans,lr)

In [24]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Ford Figo Petrol', 'Hyundai Creta 1.6', 'Mahindra Jeep CL550',
       'Mahindra Jeep MM', 'Maruti Suzuki Swift', 'Maruti Suzuki Zen',
       'Nissan X Trail', 'Tata Tiago Revotorq', 'Tata Tigor Revotron',
       'Toyota Innova 2.5'], dtype=object),
                                                                            array(['Ford', 'Hyundai', 'Mahindra', 'Maruti', 'Nissan', 'Tata',
       'Toyota'], dtype=object),
                                                                            array(['Diesel', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [26]:
y_pred = pipe.predict(x_test)

In [27]:
r2_score(y_test,y_pred)

-0.8004388481099092

In [28]:
scores=[]
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_pred=pipe.predict(x_test)
    scores.append(r2_score(y_test,y_pred))

In [29]:
np.argmax(scores)

669

In [30]:
scores[np.argmax(scores)]

0.9233699687101327

In [43]:
pipe.predict(pd.DataFrame(columns=x_test.columns,data=np.array(['Honda City','Honda',2012,10000,'Petrol']).reshape(1,5)))

<class 'ValueError'>: Found unknown categories ['Honda City'] in column 0 during transform

In [42]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
r2_score(y_test,y_pred)

0.9233699687101327

In [35]:
import pickle


In [36]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [44]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Hyundai Santro','Hyundai',2007,10000,'Petrol']).reshape(1,5)))

<class 'ValueError'>: Found unknown categories ['Hyundai Santro'] in column 0 during transform

In [39]:
pipe.steps[0][1].transformers[0][1].categories[0]

array(['Ford Figo Petrol', 'Hyundai Creta 1.6', 'Mahindra Jeep CL550',
       'Mahindra Jeep MM', 'Maruti Suzuki Swift', 'Maruti Suzuki Zen',
       'Nissan X Trail', 'Tata Tiago Revotorq', 'Tata Tigor Revotron',
       'Toyota Innova 2.5'], dtype=object)